In [1]:
import pandas as pd

df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Documentation\Data\Main_Data.csv')
df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,Road_scraped_matches_row_ids_parsed,Label_scraped_matches_row_ids_parsed,Chain_scraped_matches_row_ids_parsed,Scraped_zipcode_to_label_match_rate,Scraped_phone_match_rate,Phone_Yelp_matches_row_ids,Yelp_phone_match_rate,place_identifier(year),Phone_Yellowpages_matches_row_ids,Yellowbook_phone_match_rate
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,"[89, 108, 172, 217, 221, 239, 264, 280, 321, 3...","[200, 270, 424, 671, 729, 876]",[],7/6 successful match,True,[357],True,1,"[0, 1, 2]",True
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,"[89, 108, 172, 217, 221, 239, 264, 280, 321, 3...","[200, 270, 424, 671, 729, 876]",[],7/6 successful match,True,[357],True,1,"[0, 1, 2]",True
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,"[89, 108, 172, 217, 221, 239, 264, 280, 321, 3...","[200, 270, 424, 671, 729, 876]",[],7/6 successful match,True,[357],True,1,"[0, 1, 2]",True
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,"[89, 108, 172, 217, 221, 239, 264, 280, 321, 3...","[200, 270, 424, 671, 729, 876]",[],7/6 successful match,True,[357],True,1,"[0, 1, 2]",True
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,"[89, 108, 172, 217, 221, 239, 264, 280, 321, 3...","[200, 270, 424, 671, 729, 876]",[],7/6 successful match,True,[357],True,1,"[0, 1, 2]",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Corning ( 96021 ) Love's Travel Stop # 410,2 530-824-8767 I-5 Exit 630 ( South Ave ),24 XL 24 24,Corning,96021,Love's Travel Stop # 410,530-824-8767,2016,Corning,CA,...,"[105, 114, 138, 141, 152, 153, 154, 155, 157, ...","[0, 3, 11, 12, 15, 16, 18, 19, 20, 21, 22, 23,...","[34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 4...",7/6 successful match,True,[29],True,505,"[1308, 1309, 1310]",True
2329,", 96021 Flying J Travel Plaza # 0510154",CCorning 2530-824-8770 I-5 Exit 630 ( South Ave ),XL 24 45,NaN,96021,Flying J Travel Plaza # 0510154,530-824-8770,2007,Corning,CA,...,"[105, 114, 138, 141, 152, 153, 154, 155, 157, ...","[0, 3, 15, 16, 17, 18, 19, 20, 21, 22, 23, 27,...","[17, 18, 19, 20, 21, 22, 23, 156, 157, 158, 15...",6/6 successful match,False,NaN,False,506,[1311],False
2330,"Corning , 96021 Flying J Travel Plaza # 0510154",2 530-824-8770 I-5 Exit 630 ( South Ave ),124 HRS XL 24 HRS,Corning,96021,Flying J Travel Plaza # 0510154,530-824-8770,2008,Corning,CA,...,"[105, 114, 138, 141, 152, 153, 154, 155, 157, ...","[0, 3, 15, 16, 17, 18, 19, 20, 21, 22, 23, 27,...","[17, 18, 19, 20, 21, 22, 23, 156, 157, 158, 15...",6/6 successful match,False,NaN,False,506,[1311],False
2331,A Grenada ( 96038 ) 3 J's Food Mart ( 76 ),530-436-2208 I-5 Exit 766 ( CA A12 E ),<U+2610><U+2610> XL <U+2610>,Grenada,96038,3 J's Food Mart ( 76 ),530-436-2208,2014,Grenada,CA,...,"[105, 114, 138, 141, 152, 153, 154, 155, 157, ...","[25, 54, 55, 56, 57, 58, 115, 133, 141, 144, 1...",[],7/6 successful match,False,[321],True,506,"[1312, 1313, 1314]",True


In [2]:
condition = (
    ((df['Scraped_phone_match_rate'] == True) | 
     (df['Scraped_zipcode_to_label_match_rate'] == '7/6 successful match')) |
    (df['Yelp_phone_match_rate'] == True)|
    (df['Yellowbook_phone_match_rate'] == True)
)

In [3]:
# For each row, extract rows from Webscraped_Data.csv using Phone_scraped_matches_row_ids, prefix columns with 'Webscraped_' and join to df
import ast
webscraped_df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Documentation\Data\Webscraped_Data.csv')

# Function to extract and prefix columns for each row
def get_webscraped_data(row):
    try:
        # Parse the string representation of list to actual list
        idx_list = ast.literal_eval(row['Phone_scraped_matches_row_ids'])
        if not isinstance(idx_list, list):
            return pd.Series()
        # Extract rows from webscraped_df by index
        selected = webscraped_df.iloc[idx_list]
        # Prefix columns and flatten (if multiple, join with ';')
        prefixed = {}
        for col in webscraped_df.columns:
            values = selected[col].astype(str).tolist()
            prefixed['Webscraped_Phone_' + col] = '; '.join(values)
        return pd.Series(prefixed)
    except Exception as e:
        return pd.Series()

webscraped_cols = df.apply(get_webscraped_data, axis=1)

df = pd.concat([df, webscraped_cols], axis=1)

df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,Webscraped_Phone_Fax,Webscraped_Phone_Phone 3,Webscraped_Phone_Mile Marker,Webscraped_Phone_Phone 4,Webscraped_Phone_Mailing Address,Webscraped_Phone_Phone 5,Webscraped_Phone_Road Name,Webscraped_Phone_https,Webscraped_Phone_htp,Webscraped_Phone_http
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Corning ( 96021 ) Love's Travel Stop # 410,2 530-824-8767 I-5 Exit 630 ( South Ave ),24 XL 24 24,Corning,96021,Love's Travel Stop # 410,530-824-8767,2016,Corning,CA,...,530-824-8776; 530-824-8776,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; www.loves.com/locations/410,nan; nan,nan; nan
2329,", 96021 Flying J Travel Plaza # 0510154",CCorning 2530-824-8770 I-5 Exit 630 ( South Ave ),XL 24 45,NaN,96021,Flying J Travel Plaza # 0510154,530-824-8770,2007,Corning,CA,...,,,,,,,,,,
2330,"Corning , 96021 Flying J Travel Plaza # 0510154",2 530-824-8770 I-5 Exit 630 ( South Ave ),124 HRS XL 24 HRS,Corning,96021,Flying J Travel Plaza # 0510154,530-824-8770,2008,Corning,CA,...,,,,,,,,,,
2331,A Grenada ( 96038 ) 3 J's Food Mart ( 76 ),530-436-2208 I-5 Exit 766 ( CA A12 E ),<U+2610><U+2610> XL <U+2610>,Grenada,96038,3 J's Food Mart ( 76 ),530-436-2208,2014,Grenada,CA,...,,,,,,,,,,


In [4]:
# Compute Webscraped_Place_Match based on the described logic
import ast

def parse_list(val):
    try:
        out = ast.literal_eval(val) if isinstance(val, str) else val
        return out if isinstance(out, list) else []
    except:
        return []

def get_place_match(row):
    # Read all lists
    zip_list = set(parse_list(row.get('ZIP_scraped_matches_row_ids_parsed', [])))
    state_list = set(parse_list(row.get('State_scraped_matches_row_ids_parsed', [])))
    city_list = set(parse_list(row.get('City_scraped_matches_row_ids_parsed', [])))
    exit_list = set(parse_list(row.get('Exit_scraped_matches_row_ids_parsed', [])))
    road_list = set(parse_list(row.get('Road_scraped_matches_row_ids_parsed', [])))
    label_list = set(parse_list(row.get('Label_scraped_matches_row_ids_parsed', [])))
    # Union ZIP and State
    base = zip_list.union(state_list)
    # Eliminate numbers not found in City or Exit
    city_exit = city_list.union(exit_list)
    filtered = base.intersection(city_exit)
    # Keep only those found in both Road and Label
    final = [x for x in filtered if x in road_list and x in label_list]
    return final

df['Webscraped_Place_Match'] = df.apply(get_place_match, axis=1)

df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,Webscraped_Phone_Phone 3,Webscraped_Phone_Mile Marker,Webscraped_Phone_Phone 4,Webscraped_Phone_Mailing Address,Webscraped_Phone_Phone 5,Webscraped_Phone_Road Name,Webscraped_Phone_https,Webscraped_Phone_htp,Webscraped_Phone_http,Webscraped_Place_Match
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,"[424, 876]"
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,"[424, 876]"
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,"[424, 876]"
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,"[424, 876]"
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,"[424, 876]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Corning ( 96021 ) Love's Travel Stop # 410,2 530-824-8767 I-5 Exit 630 ( South Ave ),24 XL 24 24,Corning,96021,Love's Travel Stop # 410,530-824-8767,2016,Corning,CA,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; www.loves.com/locations/410,nan; nan,nan; nan,"[678, 208, 306, 764]"
2329,", 96021 Flying J Travel Plaza # 0510154",CCorning 2530-824-8770 I-5 Exit 630 ( South Ave ),XL 24 45,NaN,96021,Flying J Travel Plaza # 0510154,530-824-8770,2007,Corning,CA,...,,,,,,,,,,"[678, 208, 306, 764]"
2330,"Corning , 96021 Flying J Travel Plaza # 0510154",2 530-824-8770 I-5 Exit 630 ( South Ave ),124 HRS XL 24 HRS,Corning,96021,Flying J Travel Plaza # 0510154,530-824-8770,2008,Corning,CA,...,,,,,,,,,,"[678, 208, 306, 764]"
2331,A Grenada ( 96038 ) 3 J's Food Mart ( 76 ),530-436-2208 I-5 Exit 766 ( CA A12 E ),<U+2610><U+2610> XL <U+2610>,Grenada,96038,3 J's Food Mart ( 76 ),530-436-2208,2014,Grenada,CA,...,,,,,,,,,,[]


In [5]:
# For each row, extract rows from Webscraped_Data.csv using Webscraped_Place_Match, prefix columns with 'Webscraped_PlacedMatched_' and join to df
def get_webscraped_place_matched_data(row):
    try:
        idx_list = row['Webscraped_Place_Match']
        if not isinstance(idx_list, list):
            return pd.Series()
        selected = webscraped_df.iloc[idx_list]
        prefixed = {}
        for col in webscraped_df.columns:
            values = selected[col].astype(str).tolist()
            prefixed['Webscraped_PlacedMatched_' + col] = '; '.join(values)
        return pd.Series(prefixed)
    except Exception as e:
        return pd.Series()

webscraped_place_matched_cols = df.apply(get_webscraped_place_matched_data, axis=1)

df = pd.concat([df, webscraped_place_matched_cols], axis=1)

df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,Webscraped_PlacedMatched_Fax,Webscraped_PlacedMatched_Phone 3,Webscraped_PlacedMatched_Mile Marker,Webscraped_PlacedMatched_Phone 4,Webscraped_PlacedMatched_Mailing Address,Webscraped_PlacedMatched_Phone 5,Webscraped_PlacedMatched_Road Name,Webscraped_PlacedMatched_https,Webscraped_PlacedMatched_htp,Webscraped_PlacedMatched_http
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan,nan; nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Corning ( 96021 ) Love's Travel Stop # 410,2 530-824-8767 I-5 Exit 630 ( South Ave ),24 XL 24 24,Corning,96021,Love's Travel Stop # 410,530-824-8767,2016,Corning,CA,...,530-824-8776; 530-824-8776; 530-824-0547; 530-...,nan; nan; 530-824-4646 (SUBWAY); 530-824-4646 ...,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,www.loves.com/locations/410; nan; nan; www.ta-...,nan; nan; nan; nan,nan; nan; nan; nan
2329,", 96021 Flying J Travel Plaza # 0510154",CCorning 2530-824-8770 I-5 Exit 630 ( South Ave ),XL 24 45,NaN,96021,Flying J Travel Plaza # 0510154,530-824-8770,2007,Corning,CA,...,530-824-8776; 530-824-8776; 530-824-0547; 530-...,nan; nan; 530-824-4646 (SUBWAY); 530-824-4646 ...,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,www.loves.com/locations/410; nan; nan; www.ta-...,nan; nan; nan; nan,nan; nan; nan; nan
2330,"Corning , 96021 Flying J Travel Plaza # 0510154",2 530-824-8770 I-5 Exit 630 ( South Ave ),124 HRS XL 24 HRS,Corning,96021,Flying J Travel Plaza # 0510154,530-824-8770,2008,Corning,CA,...,530-824-8776; 530-824-8776; 530-824-0547; 530-...,nan; nan; 530-824-4646 (SUBWAY); 530-824-4646 ...,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,nan; nan; nan; nan,www.loves.com/locations/410; nan; nan; www.ta-...,nan; nan; nan; nan,nan; nan; nan; nan
2331,A Grenada ( 96038 ) 3 J's Food Mart ( 76 ),530-436-2208 I-5 Exit 766 ( CA A12 E ),<U+2610><U+2610> XL <U+2610>,Grenada,96038,3 J's Food Mart ( 76 ),530-436-2208,2014,Grenada,CA,...,,,,,,,,,,


In [6]:
# For each row, extract rows from Yelp_Data.csv using Phone_Yelp_matches_row_ids, prefix columns with 'Yelp_' and join to df
import ast
yelp_df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Documentation\Data\Yelp_Data.csv')
def get_yelp_data(row):
    try:
        idx_list = row['Phone_Yelp_matches_row_ids']
        # Parse string representation of list if needed
        if isinstance(idx_list, str):
            try:
                idx_list = ast.literal_eval(idx_list)
            except:
                return pd.Series()
        if not isinstance(idx_list, list):
            return pd.Series()
        selected = yelp_df.iloc[idx_list]
        prefixed = {}
        for col in yelp_df.columns:
            values = selected[col].astype(str).tolist()
            prefixed['Yelp_' + col] = '; '.join(values)
        return pd.Series(prefixed)
    except Exception as e:
        return pd.Series()

yelp_cols = df.apply(get_yelp_data, axis=1)

df = pd.concat([df, yelp_cols], axis=1)

df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,Yelp_City,Yelp_State,Yelp_Zip_Code,Yelp_Phone,Yelp_Categories,Yelp_Latitude,Yelp_Longitude,Yelp_Price,Yelp_Is_Closed,Yelp_URL
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,Coalville,UT,84017.0,(435) 336-4421,Campgrounds,40.91082982794845,-111.40519057258884,nan,False,https://www.yelp.com/biz/holiday-hills-rv-park...
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,Coalville,UT,84017.0,(435) 336-4421,Campgrounds,40.91082982794845,-111.40519057258884,nan,False,https://www.yelp.com/biz/holiday-hills-rv-park...
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,Coalville,UT,84017.0,(435) 336-4421,Campgrounds,40.91082982794845,-111.40519057258884,nan,False,https://www.yelp.com/biz/holiday-hills-rv-park...
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,Coalville,UT,84017.0,(435) 336-4421,Campgrounds,40.91082982794845,-111.40519057258884,nan,False,https://www.yelp.com/biz/holiday-hills-rv-park...
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,Coalville,UT,84017.0,(435) 336-4421,Campgrounds,40.91082982794845,-111.40519057258884,nan,False,https://www.yelp.com/biz/holiday-hills-rv-park...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Corning ( 96021 ) Love's Travel Stop # 410,2 530-824-8767 I-5 Exit 630 ( South Ave ),24 XL 24 24,Corning,96021,Love's Travel Stop # 410,530-824-8767,2016,Corning,CA,...,Corning,CA,96021.0,(530) 824-8767,"Gas Stations, Convenience Stores, Commercial T...",39.9076,-122.19635,$,False,https://www.yelp.com/biz/loves-travel-stop-cor...
2329,", 96021 Flying J Travel Plaza # 0510154",CCorning 2530-824-8770 I-5 Exit 630 ( South Ave ),XL 24 45,NaN,96021,Flying J Travel Plaza # 0510154,530-824-8770,2007,Corning,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2330,"Corning , 96021 Flying J Travel Plaza # 0510154",2 530-824-8770 I-5 Exit 630 ( South Ave ),124 HRS XL 24 HRS,Corning,96021,Flying J Travel Plaza # 0510154,530-824-8770,2008,Corning,CA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2331,A Grenada ( 96038 ) 3 J's Food Mart ( 76 ),530-436-2208 I-5 Exit 766 ( CA A12 E ),<U+2610><U+2610> XL <U+2610>,Grenada,96038,3 J's Food Mart ( 76 ),530-436-2208,2014,Grenada,CA,...,Grenada,CA,96038.0,(530) 436-2208,Gas Stations,41.64681069892627,-122.5283874116567,nan,False,https://www.yelp.com/biz/shell-gas-grenada?adj...


In [7]:
# For each row, extract rows from YellowPages_Data.csv using Phone_Yellowpages_matches_row_ids, prefix columns with 'YellowPages_' and join to df
import ast
yellowpages_df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Documentation\Data\YellowPages_Data.csv')
def get_yellowpages_data(row):
    try:
        idx_list = row['Phone_Yellowpages_matches_row_ids']
        # Parse string representation of list if needed
        if isinstance(idx_list, str):
            try:
                idx_list = ast.literal_eval(idx_list)
            except:
                return pd.Series()
        if not isinstance(idx_list, list):
            return pd.Series()
        selected = yellowpages_df.iloc[idx_list]
        prefixed = {}
        for col in yellowpages_df.columns:
            values = selected[col].astype(str).tolist()
            prefixed['YellowPages_' + col] = '; '.join(values)
        return pd.Series(prefixed)
    except Exception as e:
        return pd.Series()

yellowpages_cols = df.apply(get_yellowpages_data, axis=1)

df = pd.concat([df, yellowpages_cols], axis=1)

df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,YellowPages_JSONLD_PHONE_1,YellowPages_JSONLD_STATE_1,YellowPages_JSONLD_STREET_1,YellowPages_JSONLD_ZIP_1,YellowPages_ORIGINAL_PHONE,YellowPages_PHONE,YellowPages_SCRAPED_AT,YellowPages_SEARCH_URL,YellowPages_STATUS,YellowPages_WEBSITE
0,"Coalville , 84017 Holiday Hills ( 66 )",435-336-4421 I-80 Exit 162 ( UT 280 ),MO,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2006,Coalville,UT,...,(435) 336-4421; (435) 336-4421; (435) 336-4421,UT; UT; UT,200 S 200 W; 200 W 500 S; 212 S 500 W,84017.0; 84017.0; 84017.0,435-336-4421; 435-336-4421; 435-336-4421,(435) 336-4421Call; (435) 336-4421Call; (435) ...,2025-07-07T04:11:50.073Z; 2025-07-07T04:11:53....,https://www.yellowpages.com/search?search_term...,nan; nan; nan,http://www.phillips66gas.com; https://www.phil...
1,"Coalville , 84017 Hills ( 66 )",435-336-4421 I-80 Holiday Exit 162 ( UT 280 ),M <U+25A1>,Coalville,84017,Hills ( 66 ),435-336-4421,2007,Coalville,UT,...,(435) 336-4421; (435) 336-4421; (435) 336-4421,UT; UT; UT,200 S 200 W; 200 W 500 S; 212 S 500 W,84017.0; 84017.0; 84017.0,435-336-4421; 435-336-4421; 435-336-4421,(435) 336-4421Call; (435) 336-4421Call; (435) ...,2025-07-07T04:11:50.073Z; 2025-07-07T04:11:53....,https://www.yellowpages.com/search?search_term...,nan; nan; nan,http://www.phillips66gas.com; https://www.phil...
2,"D Coalville , 84017 Holiday Hills ( 66 ) )",4 435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+2610> <U+2610> <U+2610>,Coalville,84017,Holiday Hills ( 66 ) ),435-336-4421,2008,Coalville,UT,...,(435) 336-4421; (435) 336-4421; (435) 336-4421,UT; UT; UT,200 S 200 W; 200 W 500 S; 212 S 500 W,84017.0; 84017.0; 84017.0,435-336-4421; 435-336-4421; 435-336-4421,(435) 336-4421Call; (435) 336-4421Call; (435) ...,2025-07-07T04:11:50.073Z; 2025-07-07T04:11:53....,https://www.yellowpages.com/search?search_term...,nan; nan; nan,http://www.phillips66gas.com; https://www.phil...
3,D Coalville ( 84017 ) Holiday Hills ( 66 ),435-336-4421 I-80 Exit 162 ( UT 280 ),M <U+25A1> <U+2610>,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2014,Coalville,UT,...,(435) 336-4421; (435) 336-4421; (435) 336-4421,UT; UT; UT,200 S 200 W; 200 W 500 S; 212 S 500 W,84017.0; 84017.0; 84017.0,435-336-4421; 435-336-4421; 435-336-4421,(435) 336-4421Call; (435) 336-4421Call; (435) ...,2025-07-07T04:11:50.073Z; 2025-07-07T04:11:53....,https://www.yellowpages.com/search?search_term...,nan; nan; nan,http://www.phillips66gas.com; https://www.phil...
4,D Coalville ( 84017 ) Holiday Hills ( 66 ),4 435-336-4421 I-80 Exit 162 ( UT 280 ),D M <U+25A1> D D,Coalville,84017,Holiday Hills ( 66 ),435-336-4421,2015,Coalville,UT,...,(435) 336-4421; (435) 336-4421; (435) 336-4421,UT; UT; UT,200 S 200 W; 200 W 500 S; 212 S 500 W,84017.0; 84017.0; 84017.0,435-336-4421; 435-336-4421; 435-336-4421,(435) 336-4421Call; (435) 336-4421Call; (435) ...,2025-07-07T04:11:50.073Z; 2025-07-07T04:11:53....,https://www.yellowpages.com/search?search_term...,nan; nan; nan,http://www.phillips66gas.com; https://www.phil...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2328,Corning ( 96021 ) Love's Travel Stop # 410,2 530-824-8767 I-5 Exit 630 ( South Ave ),24 XL 24 24,Corning,96021,Love's Travel Stop # 410,530-824-8767,2016,Corning,CA,...,(530) 824-8767; (530) 824-8767; (530) 824-8767,CA; CA; CA,2120 South Ave; 2120 South Ave; 2120 South Ave,96021.0; 96021.0; 96021.0,530-824-8767; 530-824-8767; 530-824-8767,(530) 824-8767Call; (530) 824-8767Call; (530) ...,2025-07-07T06:22:44.040Z; 2025-07-07T06:22:47....,https://www.yellowpages.com/search?search_term...,nan; nan; nan,http://pilotflyingj.com; http://catscale.com/?...
2329,", 96021 Flying J Travel Plaza # 0510154",CCorning 2530-824-8770 I-5 Exit 630 ( South Ave ),XL 24 45,NaN,96021,Flying J Travel Plaza # 0510154,530-824-8770,2007,Corning,CA,...,nan,nan,nan,nan,530-824-8770,nan,2025-07-07T06:22:59.220Z,https://www.yellowpages.com/search?search_term...,no_resu

In [8]:
df.to_csv('1.csv', index=False)